Igual que en el ejercicio de pair programming de ETL Transformación II, tendréis que crear una clase que nos permita cargar los datos en SQL I.

In [ ]:
import pandas as pd
import mysql.connector
from datetime import datetime

In [ ]:
class Creacion_SQL:

    def __init__(self, nombre_bbdd, pwd):
        self.nombre_bbdd = nombre_bbdd
        self.pwd = pwd


    def create_DB(self):

        mydb = mysql.connector.connect(
            host="localhost",
            user="root",
            password= f'{self.pwd}' 
        )
        print("Conexión realizada con éxito")
        
        mycursor = mydb.cursor()

        try:
            mycursor.execute(f"CREATE SCHEMA IF NOT EXISTS {self.nombre_bbdd} DEFAULT CHARACTER SET utf8 ;")
            print(mycursor)
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)


    def create_insert(self, query):
    

        cnx = mysql.connector.connect(user='root', password=f"{self.pwd}",
                                        host='127.0.0.1', database=f"{self.nombre_bbdd}")

        mycursor = cnx.cursor()
        
        
        try: 
            mycursor.execute(query)
            cnx.commit() 
        
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)

    def check_fechas_comunidades(self, query):

           
            cnx = mysql.connector.connect(user='root', password=f'{self.pwd}',
                                        host='127.0.0.1', database=f"{self.nombre_bbdd}")
            mycursor = cnx.cursor()


            
            mycursor.execute(query)
            datos = mycursor.fetchall()
            return datos


    def sacar_id_comunidad(self, comunidad):
    
        cnx = mysql.connector.connect(user='root', password= f'{self.pwd}',
                                    host='127.0.0.1', database=f"{self.nombre_bbdd}",  auth_plugin = 'mysql_native_password')
        mycursor = cnx.cursor()

        try: 
            query_sacar_id_com = f"SELECT idcomunidades FROM comunidades WHERE comunidad = f'{comunidad}'"
            mycursor.execute(query_sacar_id_com)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
            return "Sorry, no tenemos esa comunidad en la BBDD y por lo tanto no te podemos dar su id. "
        
    def sacar_id_fecha(self, fecha):
    
        cnx = mysql.connector.connect(user='root', password= f'{self.pwd}',
                                    host='127.0.0.1', database=f"{self.nombre_bbdd}",  auth_plugin = 'mysql_native_password')
        mycursor = cnx.cursor()

        try: 
            query_sacar_id_fecha = f"SELECT idFechas FROM fechas WHERE fecha = f'{fecha}'"
            mycursor.execute(query_sacar_id_fecha)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
            return "Sorry, no tenemos esa fecha en la BBDD y por lo tanto no te podemos dar su id. "

In [ ]:
rompe_ralph= Creacion_SQL("ralph", "AlumnaAdalab")

In [ ]:
rompe_ralph.create_DB()

In [ ]:
tabla_fechas = """
CREATE TABLE IF NOT EXISTS `ralph`.`Fechas` (
  `idFechas` INT NOT NULL AUTO_INCREMENT,
  `fecha` DATE NOT NULL,
  PRIMARY KEY (`idFechas`))
ENGINE = InnoDB;
"""

tabla_nacional_R_nR = """
CREATE TABLE IF NOT EXISTS `ralph`.`nacional_renovable_no_renovable` (
  `idnacional_renovable_no_renovable` INT NOT NULL AUTO_INCREMENT,
  `porcentaje` INT NOT NULL,
  `tipo_energia` VARCHAR(45) NOT NULL,
  `valor` DECIMAL NOT NULL,
  `Fechas_idFechas` INT NOT NULL,
  PRIMARY KEY (`idnacional_renovable_no_renovable`),
  INDEX `fk_nacional_renovable_no_renovable_Fechas_idx` (`Fechas_idFechas` ASC) VISIBLE,
  CONSTRAINT `fk_nacional_renovable_no_renovable_Fechas`
    FOREIGN KEY (`Fechas_idFechas`)
    REFERENCES `ralph`.`Fechas` (`idFechas`)
    ON DELETE CASCADE
    ON UPDATE CASCADE)
ENGINE = InnoDB;
"""

tabla_comunidades_R_nR = """
CREATE TABLE IF NOT EXISTS `ralph`.`comunidades_renovable_no_renovable` (
  `idcomunidades_renovable_no_renovable` INT NOT NULL AUTO_INCREMENT,
  `porcentaje` INT NOT NULL,
  `tipo_energia` VARCHAR(45) NOT NULL,
  `valor` DECIMAL NOT NULL,
  `Fechas_idFechas` INT NOT NULL,
  `comunidades_idcomunidades` INT NOT NULL,
  PRIMARY KEY (`idcomunidades_renovable_no_renovable`),
  INDEX `fk_comunidades_renovable_no_renovable_Fechas1_idx` (`Fechas_idFechas` ASC) VISIBLE,
  INDEX `fk_comunidades_renovable_no_renovable_comunidades1_idx` (`comunidades_idcomunidades` ASC) VISIBLE,
  CONSTRAINT `fk_comunidades_renovable_no_renovable_Fechas1`
    FOREIGN KEY (`Fechas_idFechas`)
    REFERENCES `ralph`.`Fechas` (`idFechas`)
    ON DELETE CASCADE
    ON UPDATE CASCADE,
  CONSTRAINT `fk_comunidades_renovable_no_renovable_comunidades1`
    FOREIGN KEY (`comunidades_idcomunidades`)
    REFERENCES `ralph`.`comunidades` (`idcomunidades`)
    ON DELETE CASCADE
    ON UPDATE CASCADE)
ENGINE = InnoDB;
"""


tabla_comunidades = """
CREATE TABLE IF NOT EXISTS `ralph`.`comunidades` (
  `idcomunidades` INT NOT NULL AUTO_INCREMENT,
  `comunidad` VARCHAR(45) NOT NULL,
  PRIMARY KEY (`idcomunidades`))
ENGINE = InnoDB;
"""

In [ ]:
rompe_ralph.create_insert(tabla_fechas)
rompe_ralph.create_insert(tabla_comunidades)
rompe_ralph.create_insert(tabla_nacional_R_nR)
rompe_ralph.create_insert(tabla_comunidades_R_nR)


In [ ]:
df_renovables = pd.read_pickle("datos/tipo_energia.pkl")
df_comunidades = pd.read_pickle("datos/datos_comunidades.pkl")

In [ ]:
for indice, fila in df_renovables.iterrows(): 
    fila["fecha"] = datetime.strftime(fila["fecha"], "%Y-%m-%d") 
    query_fechas = f"""
            INSERT INTO Fechas (fecha) 
            VALUES ("{fila['fecha']}");
            """
    
    rompe_ralph.create_insert(query_fechas)

In [ ]:
for indice, fila in df_renovables.iterrows(): 
    fila["fecha"] = datetime.strftime(fila["fecha"], "%Y-%m-%d") 
    query_fechas = f"""
            INSERT INTO Fechas (fecha) 
            VALUES ("{fila['fecha']}");
            """
    
    rompe_ralph.create_insert(query_fechas)

In [ ]:
query_existe_fecha = f"""
            SELECT DISTINCT fecha FROM Fechas
            """

In [ ]:
rompe_ralph.check_fechas_comunidades(query_existe_fecha)

In [ ]:
for indice, fila in df_renovables.iterrows():
    fila["fecha"] = datetime.strftime(fila["fecha"], "%Y-%m-%d") 
    print(fila["fecha"])
    id_fecha = rompe_ralph.sacar_id_fecha(fila["fecha"])
    break

In [ ]:
id_fecha

In [ ]:
df_renovables.head()

In [ ]:
id_fecha

In [ ]:
for indice, fila in df_renovables.iterrows():
    id_fecha = rompe_ralph.sacar_id_fecha(fila["fecha"])
    
    query_datos_renovables = f"""
                INSERT INTO nacional_renovable_no_renovable (porcentaje, tipo_energia, valor, Fechas_idFechas) 
                VALUES ("{fila['percentage']}", "{fila['tipo_energia']}", "{fila['value']}", {id_fecha});
                """
    rompe_ralph.create_insert(query_datos_renovables)

In [ ]:
df_renovables.head(1)

In [ ]:
for indice, fila in df_renovables.iterrows():
    
    id_comunidad = sacar_id_comunidad(fila['comunidad'])
    id_fecha = sacar_id_fecha(fila["fecha"])
    
    query_medidas = f"""
                INSERT INTO mediciones (cloudcover, lifted_index, prec_type, prec_amount, temp2m, rh2m, weather, wind_direction, wind_speed, seeing, transparency, idtiempo, idlocalidad) 
                VALUES ({fila["cloudcover"]}, {fila['lifted_index']}, "{fila['prec_type']}", {fila['prec_amount']},{fila['temp2m']},{fila['rh2m']}, "{fila['weather']}","{fila['wind10m.direction']}",{fila['wind10m.speed']}, {fila['seeing']}, {fila['transparency']}, {id_fecha}, {id_ciudad});
                """
    rompe_ralph.create_insert(query_medidas)
    